# Setup

In [2]:
import cv2
import cv2.aruco as aruco
import os
import numpy as np
import subprocess

calib_id = "05-08-25"
calib_base_folder = f"/home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/{calib_id}"
# Camera 0 refers to top camera, Camera 1 refers to bottom camera
# Ensure camera port ids are correct
# port_ids = [0, 2]
cam0_device = f"/dev/cam0"
cam1_device = f"/dev/cam1"

# Ensure proper camera configurations
cam0_focus_value = 35
cam1_focus_value = 75
config_commands = {cam0_device: [
                    f"v4l2-ctl -d {cam0_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam0_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam0_device} -c focus_absolute={cam0_focus_value}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ],
                cam1_device: [
                    f"v4l2-ctl -d {cam1_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam1_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam1_device} -c focus_absolute={cam1_focus_value}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ]
                }

def configure_camera(devices, config_commands):
    for device in devices:

        print(f"Configuring camera on {device}...")

        for command in config_commands[device]:
            subprocess.run(command, shell=True, check=True)

        print("Camera configuration complete!")

# Camera Configuration

In [4]:
import subprocess
import cv2
import time

# Configure and open cameras to check proper configs
cap0 = cv2.VideoCapture(cam0_device, cv2.CAP_V4L2)
cap1 = cv2.VideoCapture(cam1_device, cv2.CAP_V4L2)
# configure_camera([cam0_device, cam1_device], config_commands) # Uncomment to use default configs

if not cap0.isOpened() or not cap1.isOpened():
    print("Error: One or both cameras could not be opened.")
    exit()

cam0_focus_adjust = cam0_focus_value
cam1_focus_adjust = cam1_focus_value
# configure_wait_time = 2 # wait this many seconds to configure manual settings
# configure_yet = False
# start_time = time.time()
while True:
    # if time.time() - start_time > configure_wait_time and not configure_yet:
    #     configure_camera([cam0_device, cam1_device], config_commands)
    #     configure_yet = True
    #     print("Configured manual camera settings!!!")

    # Read frames from both cameras
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()

    if not ret0 or not ret1:
        print("Error: One or both frames could not be read.")
        break

    # Display each camera feed in separate windows
    cv2.putText(frame0, f"Focus: {cam0_focus_adjust}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame1, f"Focus: {cam1_focus_adjust}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Camera 0 (top)", frame0)
    cv2.imshow("Camera 1 (side)", frame1)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord('q'):  # Increase focus for Camera 0
        cam0_focus_adjust = min(cam0_focus_adjust + 5, 250)
        command = f"v4l2-ctl -d {cam0_device} -c focus_absolute={cam0_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 0 focus set to: {cam0_focus_adjust}")
    elif key == ord('a'):  # Decrease focus for Camera 0
        cam0_focus_adjust = max(cam0_focus_adjust - 5, 0)
        command = f"v4l2-ctl -d {cam0_device} -c focus_absolute={cam0_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 0 focus set to: {cam0_focus_adjust}")
    elif key == ord('w'): # Increase focus for Camera 1
        cam1_focus_adjust = min(cam1_focus_adjust + 5, 250)
        command = f"v4l2-ctl -d {cam1_device} -c focus_absolute={cam1_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 1 focus set to: {cam1_focus_adjust}")
    elif key == ord('s'):
        # Decrease focus for Camera 1
        cam1_focus_adjust = max(cam1_focus_adjust - 5, 0)
        command = f"v4l2-ctl -d {cam1_device} -c focus_absolute={cam1_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 1 focus set to: {cam1_focus_adjust}")

# Release both cameras and close windows
cap0.release()
cap1.release()
cv2.destroyAllWindows()

# Collect Intrinsic Calibration Images

In [ ]:
import cv2
import os
import subprocess

# Make sure cameras are configures
configure_camera([cam0_device, cam1_device], config_commands) # Uncomment to use default configs
cap0 = cv2.VideoCapture(cam0_device, cv2.CAP_V4L2)
cap1 = cv2.VideoCapture(cam1_device, cv2.CAP_V4L2)

for i in range(2):
    print(f"Collecting instrinsic calibration images for camera {i}...")
    output_dir = f"{calib_base_folder}/charuco_calib_images/cam{i}"  # Directory to save images
    os.makedirs(output_dir, exist_ok=True)

    # Initialize webcam
    cap = cv2.VideoCapture(port_ids[i], cv2.CAP_V4L2)

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        exit()

    print("Press SPACE to capture image, ESC to exit.")

    image_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # Display the live feed
        cv2.imshow("Camera Feed - Press SPACE to capture, ESC to exit", frame)

        # Keyboard input
        key = cv2.waitKey(1) & 0xFF

        # Save image on SPACE key press
        if key == ord(' '):
            # Construct image filename
            image_path = os.path.join(output_dir, f"img_{image_count:03d}.png")
            # Save the frame as PNG
            cv2.imwrite(image_path, frame)
            print(f"Captured: {image_path}")
            image_count += 1

        # Exit on ESC key press
        elif key == 27:
            print("Exiting...")
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()


# Collect Stereo Calibration Images

In [ ]:
# Collect images of calibration board in both cameras frames for stereo extrinsic calibration
import cv2
import datetime
import os

output_dir = f"{calib_base_folder}/stereo_calib_images"
os.makedirs(output_dir, exist_ok=True)

# Make sure cameras are configures
configure_camera([cam0_device, cam1_device], config_commands) # Uncomment to use default configs
cap0 = cv2.VideoCapture(port_ids[0], cv2.CAP_V4L2)
cap1 = cv2.VideoCapture(port_ids[1], cv2.CAP_V4L2)

while True:
    # Read frames from both cameras
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()

    if not ret0 or not ret1:
        print("Error: One or both frames could not be read.")
        break

    # Display both camera feeds with timestamps
    # timestamp = datetime.now().strftime("%H:%M:%S.%f")
    # cv2.putText(frame1, f"Cam1 - {timestamp}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # cv2.putText(frame2, f"Cam2 - {timestamp}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Combine and display both frames
    combined = cv2.hconcat([frame0, frame1])
    cv2.imshow("Camera 0 (top) + Camera 1 (side)", combined)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord(' '):  # Space key to capture images
        # Save images with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        img0_path = f"{output_dir}/cam0_{frame_count}.png"
        img1_path = f"{output_dir}/cam1_{frame_count}.png"
        cv2.imwrite(img0_path, frame0)
        cv2.imwrite(img1_path, frame1)
        print(f"Captured images:\n - {img0_path}\n - {img1_path}")
        frame_count += 1

# Release both cameras and close windows
cap0.release()
cap1.release()
cv2.destroyAllWindows()

# Collect Camera-World Calibration Images

In [ ]:
# Collect images for finding camera-world transform
# Current approach is to use aruco marker to find camera 0 pose wrt to aruco marker
# Then camera-camera transform can be used to find camera 1 to world transform

# Make sure cameras are configured
configure_camera([cam0_device], config_commands)
cap = cv2.VideoCapture(port_ids[0], cv2.CAP_V4L2)

output_dir = f"{calib_base_folder}/extrinsic_calib_images/cam0"
os.makedirs(output_dir, exist_ok=True)

while True:
    # Read frame from camera
    ret, frame = cap.read()

    if not ret:
        print("Error: Frame could not be read.")
        break

    # Display the frame
    cv2.imshow("Camera Feed - Press SPACE to capture, ESC to exit", frame)

    # Keyboard input
    key = cv2.waitKey(1) & 0xFF

    # Save image on SPACE key press
    if key == ord(' '):
        # Construct image filename
        image_path = os.path.join(output_dir, f"img_aruco.png")
        # Save the frame as PNG
        cv2.imwrite(image_path, frame)
        print(f"Captured: {image_path}")

    # Exit on ESC key press
    elif key == 27:
        print("Exiting...")
        break
    
cap.release()
cv2.destroyAllWindows()

# Compute Camera Calibration

## Compute Intrinsics

In [ ]:
# Compute intrinsic, stereo extrinsic, and camera to world transforms
import cv2
import numpy as np
import glob
import cv2.aruco as aruco
import os
import pickle

def calibrate_intrinsics_charuco(glob_pattern, charuco_board, aruco_dict,
                                 min_markers=20):
    '''
    Calibrates camera intrinsics using a ChArUco board.
    '''
    all_corners, all_ids, img_size = [], [], None

    for fname in glob.glob(glob_pattern):
        img = cv2.imread(fname)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img_size is None:
            img_size = img_gray.shape[::-1]

        detector = aruco.CharucoDetector(charuco_board)
        charuco_corners, charuco_ids, marker_corners, marker_ids = detector.detectBoard(img_gray)

        all_corners.append(charuco_corners)
        all_ids.append(charuco_ids)

    # 3. Calibrate
    ret, K, dist, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=all_corners,
        charucoIds=all_ids,
        board=charuco_board,
        imageSize=img_size,
        cameraMatrix=None,
        distCoeffs=None
    )
    return ret, K, dist, rvecs, tvecs

# Dict holding all camera calibration data
camera_calib_data = {
        "cam0": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": { # this is camera to world transform
                "R": None,
                "T": None
            }
        },
        "cam1": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": {
                "R": None,
                "T": None
            }
        },
        "stereo_extrinsics": { # Camera to camera transform (from cam0 to cam1)
            "R_1_0": None,
            "T_1_0": None,
        },
    }

# Define ChArUco board used
square_size = 0.006
marker_length = 0.004
board_size = (10, 10)
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
charuco_board = aruco.CharucoBoard(
    size=board_size,
    squareLength=square_size, 
    markerLength=marker_length,
    dictionary=aruco_dict
)

# Calibrate instrinsics
for i in range(2):
    intrinsic_calib_images_path = f"{calib_base_folder}/charuco_calib_images/cam{i}/*.png"
    rms_error, K, d, _, _ = calibrate_intrinsics_charuco(
        intrinsic_calib_images_path, charuco_board, aruco_dict
    )
    print(f"Charuco intrinsics for camera {i} RMS error: {rms_error:.4f}")
    print(f"Camera {i} intrinsic matrix:\n{K}")
    print(f"Camera {i} distortion coefficients:\n{d}")
    camera_calib_data[f"cam{i}"]["intrinsics"]["K"] = K
    camera_calib_data[f"cam{i}"]["intrinsics"]["d"] = d

print("Intrinsic calibration complete.")


## Compute Stereo Extrinsics

In [ ]:
import cv2
import numpy as np
import glob
import cv2.aruco as aruco
import os
import pickle

def detect_charuco_corners(image_path, board, detector):
    """Detects Charuco corners using OpenCV 4.11+ functions."""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect Aruco markers and Charuco corners
    charuco_corners, charuco_ids, _, _ = detector.detectBoard(gray)

    if charuco_ids is not None and len(charuco_ids) > 4:
        return charuco_corners, charuco_ids
    return None, None

def calculate_relative_transform(R1, T1, R2, T2):
    """Calculate the relative rotation and translation between two cameras.
    Returns R21 and T21, the rotation and translation of camera 2 in camera 1's frame."""
    R21 = np.dot(R2, R1.T)
    T21 = T2 - np.dot(R21, T1)
    return R21, T21

def get_extrinsics(image_path, K, dist, board, detector):
    """Compute extrinsics for a single image."""
    charuco_corners, charuco_ids = detect_charuco_corners(image_path, board, detector)
    if charuco_corners is not None:
        obj_points = board.getChessboardCorners()[charuco_ids.flatten()]
        ret, rvec, tvec = cv2.solvePnP(obj_points, charuco_corners, K, dist)
        if ret:
            R, _ = cv2.Rodrigues(rvec)
            return R, tvec
    return None, None

# # Load intrinsic parameters
# calib_data_file = "../camera_calibration/05-08-25/camera_calib_data.pkl"
# with open(calib_data_file, 'rb') as f:
#     calib_data = pickle.load(f)

K0 = camera_calib_data["cam0"]["intrinsics"]["K"]
dist0 = camera_calib_data["cam0"]["intrinsics"]["d"]
K1 = camera_calib_data["cam1"]["intrinsics"]["K"]
dist1 = camera_calib_data["cam1"]["intrinsics"]["d"]

# Define ChArUco board used
square_size = 0.006
marker_length = 0.004
board_size = (10, 10)
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
charuco_board = aruco.CharucoBoard(
    size=board_size,
    squareLength=square_size, 
    markerLength=marker_length,
    dictionary=aruco_dict
)

# Create the Aruco and Charuco detector objects (new in OpenCV 4.11+)
aruco_detector = aruco.ArucoDetector(aruco_dict)
charuco_detector = aruco.CharucoDetector(charuco_board)

# Paths to synchronized images
cam0_images = sorted(glob.glob(f"{calib_base_folder}/stereo_calib_images/cam0_*.png"))
cam1_images = sorted(glob.glob(f"{calib_base_folder}/stereo_calib_images/cam1_*.png"))

# Arrays to store transformations
relative_rotations = []
relative_translations = []

for img0_path, img1_path in zip(cam0_images, cam1_images):
    # Get extrinsics for both cameras using updated functions
    R0, T0 = get_extrinsics(img0_path, K0, dist0, charuco_board, charuco_detector)
    R1, T1 = get_extrinsics(img1_path, K1, dist1, charuco_board, charuco_detector)

    if R0 is not None and R1 is not None:
        # Calculate the relative transformation between cameras
        R10, T10 = calculate_relative_transform(R0, T0, R1, T1)
        relative_rotations.append(R10)
        relative_translations.append(T10)

# Average the transformations
R_avg = sum(relative_rotations) / len(relative_rotations)
T_avg = sum(relative_translations) / len(relative_translations)

# Print the averaged relative transformation
print("Transform from Camera 0 to Camera 1:")
print("Averaged Relative Rotation (R10):\n", R_avg)
print("Averaged Relative Translation (T10):\n", T_avg)

# Save the relative transformation
camera_calib_data["stereo_extrinsics"]["R_1_0"] = R_avg
camera_calib_data["stereo_extrinsics"]["T_1_0"] = T_avg


## Compute Camera-World Transform

In [3]:
import cv2
import cv2.aruco as aruco
import numpy as np
import pickle

def solve_pnp(marker_corners, marker_length, K, dist):
    """Solve PnP for a single Aruco marker."""
    half_length = marker_length / 2

    # 3D points of the marker corners in the world coordinate system (origin at one corner)
    obj_points = np.array([
        [0, 0, 0],                  # Bottom-left (origin)
        [marker_length, 0, 0],      # Bottom-right (X-axis)
        [marker_length, marker_length, 0], # Top-right
        [0, marker_length, 0]       # Top-left (Z-axis)
    ], dtype=np.float32)

    # Solve PnP to find rotation and translation
    ret, rvec, tvec = cv2.solvePnP(obj_points, marker_corners, K, dist)
    if ret:
        R, _ = cv2.Rodrigues(rvec)
        return R, tvec
    return None, None

def draw_axes(image, R, T, K, dist):
    """Draw the world frame origin and axes on the image."""
    axis_length = 0.05  # 5 cm

    # Define the 3D points for the axes
    axis_points = np.float32([
        [0, 0, 0],              # Origin
        [axis_length, 0, 0],    # X-axis (red)
        [0, axis_length, 0],    # Y-axis (green)
        [0, 0, axis_length]     # Z-axis (blue)
    ]).reshape(-1, 3)

    # Project the 3D axis points onto the image plane
    img_points, _ = cv2.projectPoints(axis_points, cv2.Rodrigues(R)[0], T, K, dist)

    # Draw the origin point
    origin = tuple(img_points[0].ravel().astype(int))
    cv2.circle(image, origin, 5, (0, 0, 255), -1)

    # Draw the X, Y, Z axes
    cv2.line(image, origin, tuple(img_points[1].ravel().astype(int)), (0, 0, 255), 3)  # X-axis (red)
    cv2.line(image, origin, tuple(img_points[2].ravel().astype(int)), (0, 255, 0), 3)  # Y-axis (green)
    cv2.line(image, origin, tuple(img_points[3].ravel().astype(int)), (255, 0, 0), 3)  # Z-axis (blue)

    # Annotate the axes
    cv2.putText(image, "X", tuple(img_points[1].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(image, "Y", tuple(img_points[2].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(image, "Z", tuple(img_points[3].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


# Load intrinsic calibration data
# # Load intrinsic parameters
calib_data_file = f"{calib_base_folder}/camera_calib_data.pkl"
with open(calib_data_file, 'rb') as f:
    camera_calib_data = pickle.load(f)

K = camera_calib_data["cam0"]["intrinsics"]["K"]
dist = camera_calib_data["cam0"]["intrinsics"]["d"]


# Aruco marker parameters
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
marker_length = 0.004  # Length of the marker side in meters

# Initialize the Aruco detector
detector = aruco.ArucoDetector(aruco_dict)

# Load image
img = cv2.imread(f"{calib_base_folder}/extrinsic_calib_images/cam0/img_aruco.png")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect the Aruco marker
corners, ids, _ = detector.detectMarkers(img_gray)
print(corners)

if ids is not None:
    R_0_w, T_0_w = solve_pnp(corners[0], marker_length, K, dist)
    if R_0_w is not None:
        print(f"Camera 0 to World Rotation:\n{R_0_w}")
        print(f"Camera 0 to World Translation:\n{T_0_w}")

        # Draw the marker and the world frame
        aruco.drawDetectedMarkers(img, corners, ids)
        draw_axes(img, R_0_w, T_0_w, K, dist)

# Compute Camera 1 to world transform using stereo extrinsics
R_1_0 = camera_calib_data["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib_data["stereo_extrinsics"]["T_1_0"]
R_1_w = np.dot(R_1_0, R_0_w)
T_1_w = np.dot(R_1_0, T_0_w) + T_1_0
print(f"Camera 1 to World Rotation:\n{R_1_w}")
print(f"Camera 1 to World Translation:\n{T_1_w}")

# save all calibration data
camera_calib_data["cam0"]["extrinsics"]["R"] = R_0_w
camera_calib_data["cam0"]["extrinsics"]["T"] = T_0_w
camera_calib_data["cam1"]["extrinsics"]["R"] = R_1_w
camera_calib_data["cam1"]["extrinsics"]["T"] = T_1_w

with open(f"{calib_base_folder}/camera_calib_data.pkl", "wb") as f:
    pickle.dump(camera_calib_data, f)

cv2.imshow("Camera 0 - Aruco Marker with World Frame", img)
cv2.imwrite(f"{calib_base_folder}/extrinsic_calib_images/cam0/img_aruco_with_axes.png", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(array([[[295., 147.],
        [307., 146.],
        [308., 159.],
        [296., 159.]]], dtype=float32),)
Camera 0 to World Rotation:
[[ 0.96241617  0.07780901  0.26019391]
 [ 0.002285    0.95572452 -0.29425401]
 [-0.27156931  0.28378936  0.91962694]]
Camera 0 to World Translation:
[[-0.00391395]
 [-0.03319903]
 [ 0.20169472]]
Camera 1 to World Rotation:
[[ 0.27576602 -0.25516868 -0.92672725]
 [-0.05900692  0.95778878 -0.28126257]
 [ 0.95939296  0.13223629  0.24905744]]
Camera 1 to World Translation:
[[-0.03173681]
 [-0.01782421]
 [ 0.22452266]]


In [ ]:
# Verify camera to world transform by projecting axes onto image
import cv2
import numpy as np
import cv2.aruco as aruco
import pickle
import os

# Load calibration data
with open(f"{calib_base_folder}/camera_calib_data.pkl", "rb") as f:
    camera_calib_data = pickle.load(f)

# Load intrinsic and extrinsic parameters for both cameras
K0 = camera_calib_data["cam0"]["intrinsics"]["K"]
dist0 = camera_calib_data["cam0"]["intrinsics"]["d"]
R_0_w = camera_calib_data["cam0"]["extrinsics"]["R"]
T_0_w = camera_calib_data["cam0"]["extrinsics"]["T"]

K1 = camera_calib_data["cam1"]["intrinsics"]["K"]
dist1 = camera_calib_data["cam1"]["intrinsics"]["d"]
R_1_w = camera_calib_data["cam1"]["extrinsics"]["R"]
T_1_w = camera_calib_data["cam1"]["extrinsics"]["T"]

# Load images for verification
img0 = cv2.imread(f"../tip_pose_images/cam0_0.png")
img1 = cv2.imread(f"../tip_pose_images/cam1_0.png")

# Draw world frame axes on Camera 0 image
draw_axes(img0, R_0_w, T_0_w, K0, dist0)
cv2.imshow("Camera 0 - World Frame Projection", img0)

# Draw world frame axes on Camera 1 image
draw_axes(img1, R_1_w, T_1_w, K1, dist1)
cv2.imshow("Camera 1 - World Frame Projection", img1)

cv2.waitKey(0)
cv2.destroyAllWindows()